# Headline Reactions Bot

This is a simple Twitter bot that retrieves a random news headline and a random reaction gif and posts the two together.

In [40]:
import IPython.display

from bs4 import BeautifulSoup
import requests
import tweepy

import os
import shutil

#SCRIPT_DIR = os.path.dirname(os.path.realpath(__file__))
SCRIPT_DIR = os.getcwd()
LOCAL_IMG_FILE = SCRIPT_DIR + '\\reaction.gif'
RECENT_TWEETS_FILE = SCRIPT_DIR + '\\recent_tweets.dat'

TWEET_BUFFER_LEN = 30

### Import credentials

In [41]:
from credentials import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

### Setup TwitterAPI Class

In [42]:
class TwitterAPI(object):

    def __init__(self): 
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        self.api = tweepy.API(auth)
    
    def tweet(self, message):
        self.api.update_status(status=message)

    def tweet_with_img(self, img_path, msg):
        self.api.update_with_media(filename=img_path, status=msg)

def get_recent_tweets():
    '''Return list of URLs to recently tweeted pets'''
    # Get recent tweets from local file if they exist
    if os.path.isfile(RECENT_TWEETS_FILE):
        with open(RECENT_TWEETS_FILE, 'r') as f:
            recent_tweets = f.read().splitlines()
    else:
        recent_tweets = []

    return recent_tweets

def update_recent_tweets(new_item):
    '''Write profile URL to list of recent tweets'''       
    recent_tweets = get_recent_tweets()
    recent_tweets.append(new_item)

    while len(recent_tweets) > TWEET_BUFFER_LEN:
        recent_tweets.pop(0)

    with open(RECENT_TWEETS_FILE, 'w') as f:
        for item in recent_tweets:
            f.write('{0}\n'.format(item))

## Get GIFs from replygif.net

[Helpful SO post on downloading images with requests](http://stackoverflow.com/questions/13137817/how-to-download-image-using-requests)

In [43]:
def get_gif():
    GIF_SITE_URL = 'http://replygif.net/random'
    res = requests.get(GIF_SITE_URL)
    soup = BeautifulSoup(res.text)
    gif_url = soup.find('img')['src'] + ".gif"
    
    res = requests.get(gif_url, stream=True)
    with open(LOCAL_IMG_FILE, 'wb') as f:
        shutil.copyfileobj(res.raw, f)

### Get a gif

Twitter has file size limits, so the gif needs to be less than 3 MB

In [52]:
get_gif()
gif_size = os.path.getsize(LOCAL_IMG_FILE)

while gif_size > 3E6:
    get_gif()
    gif_size = os.path.getsize(LOCAL_IMG_FILE)


## Get Headlines from Reddit's /r/news


Using [PRAW](https://praw.readthedocs.org/en/stable/) module


In [46]:
import praw
import random

USER_AGENT = "Headline Gifs Twitter Account"
MAX_LEN = 120

In [47]:
submission_list = []
r = praw.Reddit(user_agent=USER_AGENT)
submissions = r.get_subreddit('news').get_top(limit=30)
for submission in submissions:
    submission_list.append(submission)

#### Find suitable headline and URL

In [48]:
recent_headlines = get_recent_tweets()
for submission in submission_list:
    headline = submission.title
    if (len(headline) < MAX_LEN) and (headline not in recent_headlines):
        cur_headline = headline
        headline_url = submission.url
        break

#### Shorten URL

In [49]:
from credentials import BITLY_TOKEN
import requests
shorten_url = 'https://api-ssl.bitly.com/v3/shorten'
payload = {'access_token': BITLY_TOKEN, 'longUrl': headline_url, 'domain':'bit.ly'}
res = requests.get(shorten_url, params = payload)
short_url = res.json()['data']['url']

## Compose and Post Tweet

In [53]:
twitter = TwitterAPI()

tweet_msg = cur_headline + ' ' + short_url
twitter.tweet_with_img(LOCAL_IMG_FILE, tweet_msg)
update_recent_tweets(cur_headline)